In [13]:
import boto3
import pandas as pd
import numpy as np
import configparser
import psycopg2
import hashlib
import uuid
from faker import Faker
import random

# Config setup

In [15]:
config = configparser.ConfigParser()
config.read('transactional_config.cfg')

['transactional_config.cfg']

# AWS client connection

In [16]:
access_key = config.get('IAM', 'ACCESS_KEY')
secret_key = config.get('IAM', 'SECRET_KEY')
region = config.get('REGION', 'REGION_NAME')

aws_rds_conn = boto3.client('rds', 
                            aws_access_key_id=access_key, 
                            aws_secret_access_key=secret_key, 
                            region_name=region)

# AWS RDS creation

In [17]:
db_instance_id = config.get('DB', 'DB_INSTANCE_ID')
db_name = config.get('DB', 'DB_NAME')
db_username = config.get('DB', 'DB_USERNAME')
db_password = config.get('DB', 'DB_PASSWORD')
db_port = config.get('DB', 'DB_PORT')
vpc_security_group = config.get('VPC', 'SECURITY_GROUP')

In [18]:
try:
    response = aws_rds_conn.create_db_instance(
        DBInstanceIdentifier=db_instance_id,
        DBName=db_name,
        MasterUsername=db_username,
        MasterUserPassword=db_password,
        Port=int(db_port),
        DBInstanceClass='db.t3.micro',
        Engine='postgres',
        PubliclyAccessible=True,
        AllocatedStorage=20,
        VpcSecurityGroupIds=[vpc_security_group]
    )
    print(response)
except aws_rds_conn.exceptions.DBInstanceAlreadyExistsFault:
    print('La instancia ya existe')
except Exception as ex:
    print(f"Error: ${ex}")

La instancia ya existe


# RDS host name

In [19]:
try:
    instance = aws_rds_conn.describe_db_instances(DBInstanceIdentifier=db_instance_id)
    RDS_HOST_NAME = instance.get('DBInstances')[0].get('Endpoint').get('Address')
    print(RDS_HOST_NAME)
except Exception as ex:
    print('Error: ', ex)

shop.cbcgkqa406pn.us-east-2.rds.amazonaws.com


# Clean Step (only if it had been run before)

In [20]:
import clean_step

try:
    db_pg_conn = psycopg2.connect(
        database=db_name,
        user=db_username,
        password=db_password,
        port=db_port,
        host=RDS_HOST_NAME
    )
    cursor = db_pg_conn.cursor()
    cursor.execute(clean_step.SQL_QUERY_DELETE_RESET)
    db_pg_conn.commit()
    print('clean step finished')
except Exception as ex:
    print('Error: ', ex)

clean step finished


# Creation of transactional system

In [21]:
import create_transactional

try:
    db_pg_conn = psycopg2.connect(
        database=db_name,
        user=db_username,
        password=db_password,
        port=db_port,
        host=RDS_HOST_NAME
    )
    cursor = db_pg_conn.cursor()
    cursor.execute(create_transactional.DDL_QUERY)
    db_pg_conn.commit()
    print('transactional tables created')
except Exception as ex:
    print('Error: ', ex)

transactional tables created


# Faker helper functions

In [23]:
fake = Faker()

# Generar datos para "categoria"
categorias_con_descripciones = {
    'Electrónica': 'Dispositivos y gadgets tecnológicos de última generación.',
    'Ropa Hombres': 'Moda y accesorios para el vestuario masculino.',
    'Ropa Mujeres': 'Colecciones y tendencias de moda femenina.',
    'Juguetes': 'Juguetes educativos y de entretenimiento para niños de todas las edades.',
    'Alimentos': 'Productos alimenticios frescos y procesados para tu despensa.',
    'Bebidas': 'Variedad de bebidas refrescantes, alcohólicas y no alcohólicas.',
    'Libros': 'Literatura, ficción, educativos y más géneros para todos los lectores.',
    'Artículos de Oficina': 'Suministros y mobiliario para mejorar tu espacio de trabajo.',
    'Deportes': 'Equipamiento y ropa deportiva para todas tus actividades físicas.',
    'Zapatos': 'Calzado cómodo y estilizado para hombres, mujeres y niños.',
    'Accesorios': 'Complementos perfectos para realzar cualquier atuendo.',
    'Belleza y Cuidado Personal': 'Productos para cuidado de la piel, cabello y belleza general.',
    'Hogar y Cocina': 'Artículos para equipar y decorar tu hogar y cocina.',
    'Jardinería': 'Herramientas y decoraciones para mantener tu jardín hermoso.',
    'Mascotas': 'Alimentos, juguetes y accesorios para el bienestar de tus mascotas.',
    'Automóviles': 'Accesorios, herramientas y tecnología para tu coche.',
    'Música': 'Instrumentos musicales y accesorios para aficionados y profesionales.',
    'Videojuegos': 'Consolas, videojuegos y accesorios para gamers.',
    'Fotografía': 'Cámaras, lentes y accesorios para fotógrafos.',
    'Celulares y Accesorios': 'Los últimos smartphones y complementos para estar siempre conectado.',
    'Computación': 'Hardware, software y periféricos para tus necesidades informáticas.',
    'Audio y Video': 'Equipos de sonido y video para una experiencia multimedia completa.',
    'Salud y Bienestar': 'Productos para mantener un estilo de vida saludable y activo.',
    'Joyería': 'Joyas finas y relojería para ocasiones especiales.',
    'Relojes': 'Una amplia selección de relojes para cada estilo y ocasión.'
}

def generar_categorias():
    return [
        {
            'nombre': nombre, 
            'descripcion': descripcion,
            'estado': True
        } for nombre, descripcion in categorias_con_descripciones.items()
    ]

# Generar datos para "rol"
roles_tiendas = [
    ('Administrador', 'Responsable de gestionar la tienda, incluidos productos, personal y finanzas'),
    ('Vendedor', 'Encargado de las ventas y la atención al cliente en el piso de ventas'),
    ('Cajero', 'Gestiona las transacciones de pago y emite recibos a los clientes'),
    ('Gerente de Producto', 'Supervisa la selección, precios y promoción de los productos'),
    ('Soporte Técnico', 'Proporciona asistencia técnica para productos electrónicos y software'),
    ('Gerente de Marketing', 'Desarrolla estrategias de marketing y publicidad para la tienda'),
    ('Gerente de RR.HH.', 'Administra la contratación, formación y políticas de recursos humanos'),
]
def generar_roles():
    return [
        {
            'nombre': rol[0], 
            'descripcion': rol[1],
            'estado': True
        } for rol in roles_tiendas
    ]


# Generar datos para "articulo"
def generar_articulo(idcategoria):
    return {
        'idcategoria': idcategoria,
        'codigo': fake.bothify(text='???-###'),
        'nombre': fake.word(),
        'precio_venta': round(random.uniform(10, 1000), 2),
        'stock': fake.random_int(min=0, max=100),
        'descripcion': fake.text(max_nb_chars=50),
        'imagen': fake.file_name(category='image'),
        'estado': fake.boolean()
    }

# Generar datos para "persona"
def generar_persona():
    tipo_persona = random.choice(['cliente', 'proveedor'])
    tipo_documento = random.choice(['DNI', 'CE', 'PAS'])
    return {
        'tipo_persona': tipo_persona,
        'nombre': fake.name()[:100],
        'tipo_documento': tipo_documento,
        'num_documento': fake.bothify(text='########'),
        'direccion': fake.address()[:70],
        'telefono': fake.phone_number()[:20],
        'email': fake.email()
    }


# Generar datos para "usuario"
def generar_usuario(idrol):
    tipo_documento = random.choice(['DNI', 'CE', 'PAS'])
    return {
        'idrol': idrol,
        'nombre': fake.name()[:100],
        'tipo_documento': tipo_documento[:20],
        'num_documento': fake.bothify(text='########')[:20],
        'direccion': fake.address()[:70],
        'telefono': fake.phone_number()[:20],
        'email': fake.email()[:50],
        'clave': fake.binary(length=16),
        'estado': fake.boolean()
    }

# Generar datos para "venta"
def generar_venta(idventa, idcliente, idusuario, total, num_serie, num_comprobante):
    tipo_comprobante = random.choice(['Factura', 'Boleta'])
    return {
        'idventa': idventa,
        'idcliente': idcliente,
        'idusuario': idusuario,
        'tipo_comprobante': tipo_comprobante,
        'serie_comprobante': num_serie,
        'num_comprobante': num_comprobante,
        'fecha': fake.date_time_this_year(),
        'impuesto': round(total * 0.07, 2),
        'total': total,
        'estado': random.choice(['Pagado', 'Pendiente', 'Anulado'])
    }

# Generar datos para "detalle_venta"
def generar_detalle_venta(idventa, idarticulo):
    return {
        'idventa': idventa,
        'idarticulo': idarticulo,
        'cantidad': fake.random_int(min=1, max=10),
        'precio': round(random.uniform(10, 100), 2),
        'descuento': round(random.uniform(0, 10), 2)
    }

# Generar datos para "ingreso"
def generar_ingreso(idingreso, idproveedor, idusuario, total, num_serie, num_comprobante):
    tipo_comprobante = random.choice(['Factura', 'Boleta', 'Ticket'])
    return {
        'idingreso': idingreso,
        'idproveedor': idproveedor,
        'idusuario': idusuario,
        'tipo_comprobante': tipo_comprobante,
        'serie_comprobante': num_serie,
        'num_comprobante': num_comprobante,
        'fecha': fake.date_time_this_year(),
        'impuesto': round(total * 0.07, 2),
        'total': total,
        'estado': random.choice(['Recibido', 'Pendiente', 'Anulado'])
    }

# Generar datos para "detalle_ingreso"
def generar_detalle_ingreso(idingreso, idarticulo):
    return {
        'idingreso': idingreso,
        'idarticulo': idarticulo,
        'cantidad': fake.random_int(min=1, max=100),
        'precio': round(random.uniform(10, 100), 2)
    }

# Insertar datos
def insertData2SQL(data_dict, table_name, driver):
    df_data = pd.DataFrame.from_records(data_dict)
    try:
        response = df_data.to_sql(table_name, driver, index=False, if_exists='append')
        print(f"Se han insertado {response} nuevos registros")
    except Exception as ex:
        print('Error: ', ex)


# Cargando datos

In [24]:
driver = f"""postgresql://{db_username}:{db_password}@{RDS_HOST_NAME}:{db_port}/{db_name}"""

In [25]:
# categorias
insertData2SQL(generar_categorias(), 'categoria', driver)

Se han insertado 25 nuevos registros


In [26]:
# roles
insertData2SQL(generar_roles(), 'rol', driver)

Se han insertado 7 nuevos registros


In [27]:
insertData2SQL([generar_persona() for i in range(300)], 'persona', driver)

Se han insertado 300 nuevos registros


In [28]:
sql_query = '''SELECT * FROM categoria;'''
df_categoria = pd.read_sql(sql_query, driver)

In [29]:
insertData2SQL([generar_articulo(random.sample(df_categoria.values.tolist(), 1)[0][0]) for i in range(1000)], 'articulo', driver)

Se han insertado 1000 nuevos registros


In [30]:
sql_query = '''SELECT * FROM rol;'''
df_rol = pd.read_sql(sql_query, driver)

In [31]:
insertData2SQL([generar_usuario(random.sample(df_rol.values.tolist(), 1)[0][0]) for i in range(1000)], 'usuario', driver)

Se han insertado 1000 nuevos registros


In [32]:
# Obtenemos articulos
sql_query = '''SELECT idarticulo FROM articulo;''';
df_articulos = pd.read_sql(sql_query, driver)

# Obtenemos personas de tipo cliente
sql_query = '''SELECT idpersona FROM persona WHERE tipo_persona = 'cliente';''';
df_clientes = pd.read_sql(sql_query, driver)

# Obtenemos usuarios (activos e inactivos)
sql_query = '''SELECT idusuario FROM usuario WHERE idrol not in (5,6,7);''';
df_usuarios = pd.read_sql(sql_query, driver)

cant_ventas = 5000
detalles_venta = []

def generar_detalles_para_venta(id_venta, df_articulos):
    num_articulos = random.randint(1, 15)
    articulos_seleccionados = random.sample(df_articulos['idarticulo'].tolist(), num_articulos)
    detalles_venta = [generar_detalle_venta(id_venta, id_articulo) for id_articulo in articulos_seleccionados]
    return detalles_venta

def generate_short_uuid(length = 10):
    uuid_str = str(uuid.uuid4())
    sha1_hash = hashlib.sha1(uuid_str.encode())
    return sha1_hash.hexdigest()[:length]

for id_venta_fake in range(cant_ventas): 
    id_venta_fake += 1
    detalles_venta.append(generar_detalles_para_venta(id_venta_fake, df_articulos))
    
detalles_venta_insertar = []
ventas = []
    
for detalle_venta in detalles_venta:
    total = 0
    for detalle in detalle_venta:
        total += detalle['precio'] - detalle['descuento']
    total = round(total, 2)
    
    # cliente y usuario aleatorio
    idcliente = lambda: random.sample(df_clientes.values.tolist(), 1)[0][0]
    idusuario = lambda: random.sample(df_usuarios.values.tolist(), 1)[0][0]
    
    # creación de objeto venta con numero de serie y correlativo con uuid
    venta = generar_venta(detalle_venta[0]['idventa'], idcliente(), idusuario(), total, generate_short_uuid(7), generate_short_uuid(10))
    ventas.append(venta)
    
    detalles_venta_insertar.extend(detalle_venta)

insertData2SQL(ventas, 'venta', driver)
insertData2SQL(detalles_venta_insertar, 'detalle_venta', driver)

Se han insertado 1000 nuevos registros
Se han insertado 450 nuevos registros


In [33]:
# Obtenemos articulos
sql_query = '''SELECT idarticulo FROM articulo;''';
df_articulos = pd.read_sql(sql_query, driver)

# Obtenemos personas de tipo proveedor
sql_query = '''SELECT idpersona FROM persona WHERE tipo_persona = 'proveedor';''';
df_proveedores = pd.read_sql(sql_query, driver)

# Obtenemos usuarios (activos e inactivos)
sql_query = '''SELECT idusuario FROM usuario WHERE idrol not in (5,6,7);''';
df_usuarios = pd.read_sql(sql_query, driver)

cant_ingresos = 2000
detalles_ingreso = []


def generar_detalles_para_ingreso(id_ingreso, df_articulos):
    num_articulos = random.randint(1, 15)
    articulos_seleccionados = random.sample(df_articulos['idarticulo'].tolist(), num_articulos)
    detalles_ingreso = [generar_detalle_ingreso(id_ingreso, id_articulo) for id_articulo in articulos_seleccionados]
    return detalles_ingreso

def generate_short_uuid(length = 10):
    uuid_str = str(uuid.uuid4())
    sha1_hash = hashlib.sha1(uuid_str.encode())
    return sha1_hash.hexdigest()[:length]

for id_ingreso_fake in range(cant_ingresos): 
    id_ingreso_fake += 1
    detalles_ingreso.append(generar_detalles_para_ingreso(id_ingreso_fake, df_articulos))
    
detalles_ingreso_insertar = []
ingresos = []

for detalle_ingreso in detalles_ingreso:
    total = 0
    for ingreso in detalle_ingreso:
        total += ingreso['precio']
    total = round(total, 2)
    
    # proveedor y usuario aleatorio
    idproveedor = lambda: random.sample(df_proveedores.values.tolist(), 1)[0][0]
    idusuario = lambda: random.sample(df_usuarios.values.tolist(), 1)[0][0]
    
    
    ingreso = generar_ingreso(detalle_ingreso[0]['idingreso'], idproveedor(), idusuario(), total, generate_short_uuid(7), generate_short_uuid(10))
    ingresos.append(ingreso)
    
    detalles_ingreso_insertar.extend(detalle_ingreso)

insertData2SQL(ingresos, 'ingreso', driver)
insertData2SQL(detalles_ingreso_insertar, 'detalle_ingreso', driver)

Se han insertado 1000 nuevos registros
Se han insertado 912 nuevos registros
